<a href="https://colab.research.google.com/github/pisceno/AI-Generative/blob/main/ENACT_DataAgent_demo_5capas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import asyncio
import json

# --- Infraestructura simple de mensajes ---
class Agent:
    def __init__(self, name):
        self.name = name
        self.inbox = asyncio.Queue()
        self.running = True

    async def send(self, other, content):
        await other.inbox.put((self.name, content))

    async def stop(self):
        self.running = False

# --- MonitorAgent ---
class MonitorAgent(Agent):
    async def run(self, orch):
        while self.running:
            telemetry = {"node": "edge-1", "cpu_pct": 75, "latency_ms": 15}
            await self.send(orch, {"type": "telemetry", "data": telemetry})
            print(f"[Monitor] Enviada telemetría: {telemetry}")
            await asyncio.sleep(5)

# --- OrchestratorAgent ---
class OrchestratorAgent(Agent):
    async def run(self, ai, dev, data):
        while self.running:
            sender, msg = await self.inbox.get()
            if msg["type"] == "telemetry":
                telemetry = msg["data"]
                print(f"[Orchestrator] Recibida telemetría de {sender}: {telemetry}")
                await self.send(ai, {"type": "forecast_request", "telemetry": telemetry})
            elif msg["type"] == "forecast_response":
                decision = msg["decision"]
                print(f"[Orchestrator] Recibida decisión de AI: {decision}")
                # Consultar DataAgent antes de enviar a Dev
                await self.send(data, {"type": "data_check", "decision": decision})

            elif msg["type"] == "data_authorization":
                authorized_decision = msg["decision"]
                print(f"[Orchestrator] DataAgent autorizó la decisión: {authorized_decision}")
                await self.send(dev, {"type": "deploy_decision", "decision": authorized_decision})

# --- AIModelAgent ---
class AIModelAgent(Agent):
    async def run(self, orch):
        while self.running:
            sender, msg = await self.inbox.get()
            if msg["type"] == "forecast_request":
                telemetry = msg["telemetry"]
                decision = {"placement": "cloud", "reason": "alta carga"} if telemetry["cpu_pct"] > 70 else {"placement": "edge", "reason": "capacidad suficiente"}
                print(f"[AI] Forecast: {decision}")
                await self.send(orch, {"type": "forecast_response", "decision": decision})

# --- DataAgent ---
class DataAgent(Agent):
    async def run(self, orch):
        while self.running:
            sender, msg = await self.inbox.get()
            if msg["type"] == "data_check":
                decision = msg["decision"]
                # Simular verificación de permisos o disponibilidad de datos
                print(f"[DataAgent] Verificando acceso para decisión: {decision}")
                # En este ejemplo siempre autoriza
                await self.send(orch, {"type": "data_authorization", "decision": decision})

# --- DevAgent ---
class DevAgent(Agent):
    async def run(self):
        while self.running:
            sender, msg = await self.inbox.get()
            if msg["type"] == "deploy_decision":
                decision = msg["decision"]
                print(f"[Dev] Recibida decisión: desplegar en {decision['placement']} (razón: {decision['reason']})")

# --- MAIN ---
async def main():
    orch = OrchestratorAgent("Orchestrator")
    mon = MonitorAgent("Monitor")
    ai = AIModelAgent("AI")
    data = DataAgent("DataAgent")
    dev = DevAgent("Dev")

    tasks = [
        asyncio.create_task(mon.run(orch)),
        asyncio.create_task(orch.run(ai, dev, data)),
        asyncio.create_task(ai.run(orch)),
        asyncio.create_task(data.run(orch)),
        asyncio.create_task(dev.run()),
    ]

    await asyncio.sleep(20)   # ejecutar 20 segundos
    print(">>> Fin de demo")
    for ag in [orch, mon, ai, data, dev]:
        await ag.stop()
    for t in tasks:
        t.cancel()

await main()


[Monitor] Enviada telemetría: {'node': 'edge-1', 'cpu_pct': 75, 'latency_ms': 15}
[Orchestrator] Recibida telemetría de Monitor: {'node': 'edge-1', 'cpu_pct': 75, 'latency_ms': 15}
[AI] Forecast: {'placement': 'cloud', 'reason': 'alta carga'}
[Orchestrator] Recibida decisión de AI: {'placement': 'cloud', 'reason': 'alta carga'}
[DataAgent] Verificando acceso para decisión: {'placement': 'cloud', 'reason': 'alta carga'}
[Orchestrator] DataAgent autorizó la decisión: {'placement': 'cloud', 'reason': 'alta carga'}
[Dev] Recibida decisión: desplegar en cloud (razón: alta carga)
[Monitor] Enviada telemetría: {'node': 'edge-1', 'cpu_pct': 75, 'latency_ms': 15}
[Orchestrator] Recibida telemetría de Monitor: {'node': 'edge-1', 'cpu_pct': 75, 'latency_ms': 15}
[AI] Forecast: {'placement': 'cloud', 'reason': 'alta carga'}
[Orchestrator] Recibida decisión de AI: {'placement': 'cloud', 'reason': 'alta carga'}
[DataAgent] Verificando acceso para decisión: {'placement': 'cloud', 'reason': 'alta carg